In [1]:
import pandas as pd
import numpy as np

# Passenger Ordered List

In [2]:
flight_details= pd.read_csv("PNRB-ZZ-20231208_062017.csv")
passenger= pd.read_csv("PNRP-ZZ-20231208_111136.csv")
schedule=pd.read_csv("SCH-ZZ-20231208_035117.csv")
# flight_details

In [3]:
#input

specific_schedule_id= "SCH-ZZ-2098565"
specific_date="20240510"

In [4]:
# segment score and pax score input

external_multiplier_seg = 100
external_multiplier_pax = 50

In [5]:
#ssr cost input and loyalty cost

tags_cost ={'INFT' : 200, 'WCHR' : 200, 'WCHS' : 200, 'WCHC' : 200, 'LANG' : 200, 'CHLD' : 200, 'MAAS' : 200, 'UNMR' : 200, 'BLND' : 200, 'DEAF' : 200, 'EXST' : 200, 'MEAL' : 200, 'NSST' : 200, 'NRPS' : 200,  }
loyalty_cost = {'Platinum' : 2000, 'Gold' : 2000, 'Silver' : 2000, 'none':0 }

In [6]:
tag_columns = ['SPECIAL_NAME_CD2', 'SSR_CODE_CD1']

In [7]:
#generating departure key from input

schedule['FlightNumber']=schedule['FlightNumber'].astype(int)
selected_rows = schedule[schedule['ScheduleID'] == specific_schedule_id][['ScheduleID', 'CarrierCode', 'ArrivalAirport', 'DepartureAirport', 'FlightNumber']]
selected_rows['FlightNumber'] = selected_rows['FlightNumber'].astype(str)
selected_rows['Concatenated'] = selected_rows[['DepartureAirport', 'ArrivalAirport', 'FlightNumber']].astype(str).apply(lambda x: ''.join(x), axis=1)
selected_rows['Concatenated'] = specific_date+selected_rows['Concatenated'].replace(' ', '', regex=True)
selected_rows['Concatenated'] = selected_rows[['CarrierCode','Concatenated']].astype(str).apply(lambda x: ''.join(x), axis=1)

#specific departure key is generated and stored in variable specific_dep_key
specific_dep_key= selected_rows.at[selected_rows.index[0], "Concatenated"]
flight_details["DEP_KEY"]=flight_details["DEP_KEY"].astype(str)
#specific_dep_key+='.0'
specific_dep_key

'ZZ20240510IDRHYD2792'

In [8]:
filtered_f= flight_details[flight_details["DEP_KEY"]== specific_dep_key]
filtered_f_copy= filtered_f
# filtered_f_copy

In [9]:
filtered_f_copy['SEG_TOTAL_MULTIPLIED'] = filtered_f_copy['SEG_TOTAL'] * external_multiplier_seg
filtered_f_copy['PAX_CNT_MULTIPLIED'] = filtered_f_copy['PAX_CNT'] * external_multiplier_pax

# Create a new column with the sum of SEG_SEQ_MULTIPLIED and PAX_CNT_MULTIPLIED
filtered_f_copy['SUM_SEG_PAX'] = filtered_f_copy['SEG_TOTAL_MULTIPLIED'] + filtered_f_copy['PAX_CNT_MULTIPLIED']

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\4089213020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_f_copy['SEG_TOTAL_MULTIPLIED'] = filtered_f_copy['SEG_TOTAL'] * external_multiplier_seg
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\4089213020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_f_copy['PAX_CNT_MULTIPLIED'] = filtered_f_copy['PAX_CNT'] * external_multiplier_pax
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\4089213020.py:5: Setti

In [10]:
reclocs_list = filtered_f["RECLOC"].tolist()
filtered_passengers_list = passenger[passenger["RECLOC"].isin(reclocs_list)]
filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
filtered_passengers_list['Loyalty_score'] = filtered_passengers_list['TierLevel'].map(loyalty_cost).fillna(loyalty_cost['none'])

filtered_passengers_list['Total_score'] = filtered_passengers_list['Loyalty_score'] + filtered_passengers_list['ssr_score']
sorted_passengers=filtered_passengers_list.sort_values(by="Total_score", ascending=False)

# filtered_passengers_list
# sorted_passengers

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\3454662936.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\3454662936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\3454662936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [11]:
merged_sorted_passengers = sorted_passengers.groupby('RECLOC')['Total_score'].sum().reset_index()
# merged_sorted_passengers
merged_table = pd.merge(merged_sorted_passengers, filtered_f_copy, on='RECLOC', how='outer')
merged_table['final_score'] = merged_table['Total_score'] + merged_table['SUM_SEG_PAX']
merged_table=merged_table.drop(['Total_score', 'SEG_TOTAL_MULTIPLIED', 'PAX_CNT_MULTIPLIED' ,'SUM_SEG_PAX'], axis=1)
merge_sorted_table = merged_table.sort_values('final_score', ascending=False)

merge_sorted_table

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,COS_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ,final_score
0,BUHV67,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,1,9,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,12950.0
9,RHDO70,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,3,6,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,8800.0
6,PJSC24,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,3,8,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,8700.0
7,PSGX74,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,BusinessClass,1,4,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,8600.0
5,MCNQ94,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,4,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,7000.0
1,ERZL70,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,BusinessClass,1,3,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,4500.0
4,JISG46,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,2,2,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,4300.0
8,QXQZ78,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,1,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,4300.0
2,GQDX67,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,FirstClass,1,2,3,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,2550.0
10,SQMP46,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,FirstClass,1,4,2,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42,2500.0


In [12]:
final_recloc_list = merge_sorted_table['RECLOC'].tolist()
final_pax_cnt=merge_sorted_table['PAX_CNT'].tolist()
final_seg_seq_cnt=merge_sorted_table['SEG_SEQ'].tolist()
final_score=merge_sorted_table['final_score'].tolist()


final_pax_cnt

totalp=0

for el in final_pax_cnt:
    totalp=totalp+el

totalp


52

# Direct Flight Ordered List

In [13]:
specific_dep_key

'ZZ20240510IDRHYD2792'

In [14]:
f = pd.read_csv('INV-ZZ-20231208_041852.csv')
f['DepartureDateTime'] = pd.to_datetime(f['DepartureDateTime'])
f['ArrivalDateTime'] = pd.to_datetime(f['ArrivalDateTime'])

In [15]:
def flights_between(departure_airport,arrival_airport,flight_schedule_df):
    filtered_flights = flight_schedule_df[
        (flight_schedule_df['DepartureAirport'] == departure_airport) &
        (flight_schedule_df['ArrivalAirport'] == arrival_airport)
    ]
    return filtered_flights

In [16]:
# Function to get City Mapping like Key(Arrival City): Values(List of Departure Cities)
city_mapping = {}

# Iterate over each row in the DataFrame
for index, row in f.iterrows():
    arrival_city = row['ArrivalAirport']
    departure_city = row['DepartureAirport']

    # Check if arrival city is already in the dictionary
    if arrival_city in city_mapping:
        # Append departure city to the existing list
        city_mapping[arrival_city].append(departure_city)
    else:
        # Create a new list for the arrival city
        city_mapping[arrival_city] = [departure_city]
for key in city_mapping.keys():
    city_mapping[key] = np.unique(city_mapping[key])
city_mapping

{'HYD': array(['AMD', 'BLR', 'BOM', 'CCU', 'CJB', 'IDR', 'MAA'], dtype='<U3'),
 'AMD': array(['BLR', 'BOM', 'DEL', 'HYD', 'MAA'], dtype='<U3'),
 'MAA': array(['AMD', 'BOM', 'CCU', 'DEL', 'GOI'], dtype='<U3'),
 'CCU': array(['AMD', 'BLR', 'BOM', 'HYD', 'MAA'], dtype='<U3'),
 'BOM': array(['AMD', 'BLR', 'CCU', 'DEL', 'GAU', 'GOI', 'MAA', 'PNQ'],
       dtype='<U3'),
 'BLR': array(['AMD', 'BBI', 'BOM', 'CCU', 'DEL', 'HYD', 'JAI', 'LKO'],
       dtype='<U3'),
 'DEL': array(['AMD', 'BLR', 'BOM', 'HYD', 'JAI'], dtype='<U3'),
 'CJB': array(['HYD', 'MAA'], dtype='<U3'),
 'TRV': array(['MAA'], dtype='<U3'),
 'PNQ': array(['BOM', 'GAU', 'GOI'], dtype='<U3'),
 'IDR': array(['HYD'], dtype='<U3'),
 'LKO': array(['JAI'], dtype='<U3'),
 'GAU': array(['GOI', 'PNQ'], dtype='<U3'),
 'GOI': array(['PNQ'], dtype='<U3')}

In [17]:
b = flights_between(city_mapping['BLR'][0],'BLR',f)
b

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,PC_Oversold,PC_AvailableInventory,EC_TotalInventory,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD
892,INV-ZZ-7703162,SCH-ZZ-5156445,ZZ,ZZ20240514AMDBLR3782.0,3782.0,Boeing 787,05/14/2024,2024-05-14 15:57:00,2024-05-15 07:24:00,AMD,...,32,43,120,65,43,55,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
893,INV-ZZ-7431689,SCH-ZZ-5156445,ZZ,ZZ20240515AMDBLR3782.0,3782.0,Boeing 787,05/15/2024,2024-05-15 15:57:00,2024-05-16 07:24:00,AMD,...,33,42,125,66,42,59,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
894,INV-ZZ-1603837,SCH-ZZ-5156445,ZZ,ZZ20240517AMDBLR3782.0,3782.0,Boeing 787,05/17/2024,2024-05-17 15:57:00,2024-05-18 07:24:00,AMD,...,35,50,119,57,51,62,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
895,INV-ZZ-6060194,SCH-ZZ-5156445,ZZ,ZZ20240518AMDBLR3782.0,3782.0,Boeing 787,05/18/2024,2024-05-18 15:57:00,2024-05-19 07:24:00,AMD,...,35,49,121,58,50,63,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
896,INV-ZZ-3258972,SCH-ZZ-5156445,ZZ,ZZ20240521AMDBLR3782.0,3782.0,Boeing 787,05/21/2024,2024-05-21 15:57:00,2024-05-22 07:24:00,AMD,...,35,45,123,62,46,61,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,INV-ZZ-4855587,SCH-ZZ-4843596,ZZ,ZZ20240519AMDBLR3041.0,3041.0,Airbus A321-200,05/19/2024,2024-05-19 14:26:00,2024-05-19 18:12:00,AMD,...,28,36,93,44,36,49,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1199,INV-ZZ-4725843,SCH-ZZ-4843596,ZZ,ZZ20240520AMDBLR3041.0,3041.0,Airbus A321-200,05/20/2024,2024-05-20 14:26:00,2024-05-20 18:12:00,AMD,...,27,36,92,45,35,47,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1200,INV-ZZ-1805422,SCH-ZZ-4843596,ZZ,ZZ20240524AMDBLR3041.0,3041.0,Airbus A321-200,05/24/2024,2024-05-24 14:26:00,2024-05-24 18:12:00,AMD,...,26,33,92,46,34,46,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1201,INV-ZZ-6244039,SCH-ZZ-4843596,ZZ,ZZ20240526AMDBLR3041.0,3041.0,Airbus A321-200,05/26/2024,2024-05-26 14:26:00,2024-05-26 18:12:00,AMD,...,29,36,90,41,39,49,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."


In [18]:
print(type(b.iloc[3]))

<class 'pandas.core.series.Series'>


In [19]:
import pandas as pd

def arrival_delay_cost_function(time_diff):
    if(time_diff <= pd.Timedelta('6 hours')):
        return 70
    elif(time_diff <= pd.Timedelta('12 hours')):
        return 50
    elif(time_diff <= pd.Timedelta('24 hours')):
        return 40
    elif(time_diff <= pd.Timedelta('48 hours')):
        return 30
    else:
        return 0
def departure_delay_cost_function(time_diff):
    if(time_diff <= pd.Timedelta('6 hours')):
        return 70
    elif(time_diff <= pd.Timedelta('12 hours')):
        return 50
    elif(time_diff <= pd.Timedelta('24 hours')):
        return 40
    elif(time_diff <= pd.Timedelta('48 hours')):
        return 30
    else:
        return 0
        
def alternate_direct_flights(cancelled_flight_dep_key, df):
    cancelled_flight = df[df['Dep_Key'] == cancelled_flight_dep_key+'.0'].iloc[0]
    flights_list = flights_between(cancelled_flight['DepartureAirport'], cancelled_flight['ArrivalAirport'], df).copy()
    
    flights_list['DepartureDateTimeDiff'] = [i-cancelled_flight['DepartureDateTime'] for i in flights_list['DepartureDateTime']]
    flights_list['ArrivalDateTimeDiff'] = [i-cancelled_flight['ArrivalDateTime'] for i in flights_list['ArrivalDateTime']]

        
    flights_list_ = flights_list.copy().iloc[0:0]
    for i in range(len(flights_list)):
        if(((flights_list.iloc[i].DepartureDateTimeDiff) >= pd.Timedelta('0 days')) and ((flights_list.iloc[i].ArrivalDateTimeDiff) >= pd.Timedelta('0 days'))):
            if(((flights_list.iloc[i].DepartureDateTimeDiff) <= pd.Timedelta('2 days')) and ((flights_list.iloc[i].ArrivalDateTimeDiff) <= pd.Timedelta('2 days'))):
                flights_list_.loc[len(flights_list_)]=flights_list.iloc[i]

    flights_list_['Cost'] = 0
    for i in range(len(flights_list_)):
        flights_list_['Cost'].iloc[i] += arrival_delay_cost_function(flights_list_['ArrivalDateTimeDiff'].iloc[i])
        flights_list_['Cost'].iloc[i] += departure_delay_cost_function(flights_list_['DepartureDateTimeDiff'].iloc[i])
        flights_list_['Cost'].iloc[i] += 40 # Same City Pairs
    return flights_list_.sort_values(by='Cost', ascending=False).iloc[1:]

# Assuming y and f are defined somewhere
x = alternate_direct_flights(specific_dep_key, f)
x

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\1152031819.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_list_['Cost'].iloc[i] += arrival_delay_cost_function(flights_list_['ArrivalDateTimeDiff'].iloc[i])
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\1152031819.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_list_['Cost'].iloc[i] += departure_delay_cost_function(flights_list_['DepartureDateTimeDiff'].iloc[i])
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_34444\1152031819.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,DepartureDateTimeDiff,ArrivalDateTimeDiff,Cost
2,INV-ZZ-7767976,SCH-ZZ-6755803,ZZ,ZZ20240511IDRHYD3455.0,3455.0,Boeing 777,05/11/2024,2024-05-11 03:41:00,2024-05-12 01:59:00,IDR,...,107,63,84,"{'F': 25, 'P': 17}","{'C': 34, 'J': 26, 'Z': 26}","{'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, ...","{'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G'...",0 days 15:19:00,1 days 11:47:00,110
1,INV-ZZ-5004957,SCH-ZZ-2098565,ZZ,ZZ20240512IDRHYD2792.0,2792.0,BAe 146,05/12/2024,2024-05-12 12:22:00,2024-05-12 14:12:00,IDR,...,20,16,20,"{'F': 5, 'P': 4}","{'C': 7, 'J': 5, 'Z': 5}","{'Q': 8, 'R': 5, 'S': 3, 'T': 3, 'H': 5, 'M': 3}","{'Y': 3, 'A': 3, 'B': 1, 'D': 1, 'E': 1, 'G': ...",2 days 00:00:00,2 days 00:00:00,100


In [21]:
(x.AvailableInventory).tolist()

[209, 52]

In [22]:
(x.Cost)

2    110
1    100
Name: Cost, dtype: int64

In [23]:
a=(x.AvailableInventory).tolist()
ac=(x.Cost).tolist()
pc=final_score
p=final_pax_cnt

m=len(a)
n=len(p)



In [24]:
pas_in_fli = [[f'pas_{i}_in_fli_{j}' for j in range(m)] for i in range(n)]
pas_in_fli

[['pas_0_in_fli_0', 'pas_0_in_fli_1'],
 ['pas_1_in_fli_0', 'pas_1_in_fli_1'],
 ['pas_2_in_fli_0', 'pas_2_in_fli_1'],
 ['pas_3_in_fli_0', 'pas_3_in_fli_1'],
 ['pas_4_in_fli_0', 'pas_4_in_fli_1'],
 ['pas_5_in_fli_0', 'pas_5_in_fli_1'],
 ['pas_6_in_fli_0', 'pas_6_in_fli_1'],
 ['pas_7_in_fli_0', 'pas_7_in_fli_1'],
 ['pas_8_in_fli_0', 'pas_8_in_fli_1'],
 ['pas_9_in_fli_0', 'pas_9_in_fli_1'],
 ['pas_10_in_fli_0', 'pas_10_in_fli_1'],
 ['pas_11_in_fli_0', 'pas_11_in_fli_1'],
 ['pas_12_in_fli_0', 'pas_12_in_fli_1'],
 ['pas_13_in_fli_0', 'pas_13_in_fli_1'],
 ['pas_14_in_fli_0', 'pas_14_in_fli_1']]

In [25]:
coe=[]

for k in range(m*n):
    coe.append(1)

coe

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [26]:
list=[]
for k in pas_in_fli:
    for n in k:
        list.append(n)

linear=dict(zip(list,coe))
linear

{'pas_0_in_fli_0': 1,
 'pas_0_in_fli_1': 1,
 'pas_1_in_fli_0': 1,
 'pas_1_in_fli_1': 1,
 'pas_2_in_fli_0': 1,
 'pas_2_in_fli_1': 1,
 'pas_3_in_fli_0': 1,
 'pas_3_in_fli_1': 1,
 'pas_4_in_fli_0': 1,
 'pas_4_in_fli_1': 1,
 'pas_5_in_fli_0': 1,
 'pas_5_in_fli_1': 1,
 'pas_6_in_fli_0': 1,
 'pas_6_in_fli_1': 1,
 'pas_7_in_fli_0': 1,
 'pas_7_in_fli_1': 1,
 'pas_8_in_fli_0': 1,
 'pas_8_in_fli_1': 1,
 'pas_9_in_fli_0': 1,
 'pas_9_in_fli_1': 1,
 'pas_10_in_fli_0': 1,
 'pas_10_in_fli_1': 1,
 'pas_11_in_fli_0': 1,
 'pas_11_in_fli_1': 1,
 'pas_12_in_fli_0': 1,
 'pas_12_in_fli_1': 1,
 'pas_13_in_fli_0': 1,
 'pas_13_in_fli_1': 1,
 'pas_14_in_fli_0': 1,
 'pas_14_in_fli_1': 1}

In [66]:
from dimod import BinaryQuadraticModel
import hybrid
from hybrid import SimulatedAnnealingProblemSampler,TabuProblemSampler,min_sample,State

bqm = BinaryQuadraticModel(linear,{},"BINARY")
lmbd = 100

In [67]:
for i in range(len(p)):
    cons1=[]
    for j in range(len(ac)):
        cons1.append((f'pas_{i}_in_fli_{j}', 1))
    print(cons1)
    bqm.add_linear_equality_constraint(cons1, lmbd, -1)

# for each passenger get 1 flight

[('pas_0_in_fli_0', 1), ('pas_0_in_fli_1', 1)]
[('pas_1_in_fli_0', 1), ('pas_1_in_fli_1', 1)]
[('pas_2_in_fli_0', 1), ('pas_2_in_fli_1', 1)]
[('pas_3_in_fli_0', 1), ('pas_3_in_fli_1', 1)]
[('pas_4_in_fli_0', 1), ('pas_4_in_fli_1', 1)]
[('pas_5_in_fli_0', 1), ('pas_5_in_fli_1', 1)]
[('pas_6_in_fli_0', 1), ('pas_6_in_fli_1', 1)]
[('pas_7_in_fli_0', 1), ('pas_7_in_fli_1', 1)]
[('pas_8_in_fli_0', 1), ('pas_8_in_fli_1', 1)]
[('pas_9_in_fli_0', 1), ('pas_9_in_fli_1', 1)]
[('pas_10_in_fli_0', 1), ('pas_10_in_fli_1', 1)]
[('pas_11_in_fli_0', 1), ('pas_11_in_fli_1', 1)]
[('pas_12_in_fli_0', 1), ('pas_12_in_fli_1', 1)]
[('pas_13_in_fli_0', 1), ('pas_13_in_fli_1', 1)]
[('pas_14_in_fli_0', 1), ('pas_14_in_fli_1', 1)]


In [68]:
for j in range(len(a)):
    cons2=[]
    
    for i in range(len(p)):
        cons2.append((f'pas_{i}_in_fli_{j}',p[i]))
    print(cons2,a[j])   
    bqm.add_linear_inequality_constraint(terms=cons2, lagrange_multiplier=lmbd/10, constant=0, label='s', lb=0, ub=a[j])
    
    # capacity constrains

[('pas_0_in_fli_0', 9), ('pas_1_in_fli_0', 6), ('pas_2_in_fli_0', 8), ('pas_3_in_fli_0', 4), ('pas_4_in_fli_0', 4), ('pas_5_in_fli_0', 4), ('pas_6_in_fli_0', 2), ('pas_7_in_fli_0', 4), ('pas_8_in_fli_0', 3), ('pas_9_in_fli_0', 2), ('pas_10_in_fli_0', 1), ('pas_11_in_fli_0', 1), ('pas_12_in_fli_0', 1), ('pas_13_in_fli_0', 1), ('pas_14_in_fli_0', 2)] 209
[('pas_0_in_fli_1', 9), ('pas_1_in_fli_1', 6), ('pas_2_in_fli_1', 8), ('pas_3_in_fli_1', 4), ('pas_4_in_fli_1', 4), ('pas_5_in_fli_1', 4), ('pas_6_in_fli_1', 2), ('pas_7_in_fli_1', 4), ('pas_8_in_fli_1', 3), ('pas_9_in_fli_1', 2), ('pas_10_in_fli_1', 1), ('pas_11_in_fli_1', 1), ('pas_12_in_fli_1', 1), ('pas_13_in_fli_1', 1), ('pas_14_in_fli_1', 2)] 52


In [69]:
sampler = SimulatedAnnealingProblemSampler()
state = State.from_sample(min_sample(bqm), bqm)
new_state = sampler.run(state).result()
solute=new_state.samples.first.sample, new_state.samples.first.energy

In [70]:
solute[0]

{'pas_0_in_fli_0': 0,
 'pas_0_in_fli_1': 1,
 'pas_10_in_fli_0': 0,
 'pas_10_in_fli_1': 1,
 'pas_11_in_fli_0': 1,
 'pas_11_in_fli_1': 0,
 'pas_12_in_fli_0': 0,
 'pas_12_in_fli_1': 1,
 'pas_13_in_fli_0': 0,
 'pas_13_in_fli_1': 1,
 'pas_14_in_fli_0': 0,
 'pas_14_in_fli_1': 1,
 'pas_1_in_fli_0': 0,
 'pas_1_in_fli_1': 1,
 'pas_2_in_fli_0': 1,
 'pas_2_in_fli_1': 0,
 'pas_3_in_fli_0': 1,
 'pas_3_in_fli_1': 0,
 'pas_4_in_fli_0': 0,
 'pas_4_in_fli_1': 1,
 'pas_5_in_fli_0': 0,
 'pas_5_in_fli_1': 1,
 'pas_6_in_fli_0': 1,
 'pas_6_in_fli_1': 0,
 'pas_7_in_fli_0': 1,
 'pas_7_in_fli_1': 0,
 'pas_8_in_fli_0': 0,
 'pas_8_in_fli_1': 1,
 'pas_9_in_fli_0': 0,
 'pas_9_in_fli_1': 1}

In [71]:
no_pas=0

for i in range(len(p)):
    for j in range(len(a)):
     if solute[0][f'pas_{i}_in_fli_{j}']==1:
        no_pas=no_pas+1
no_pas

15

In [72]:
fli_cap=[]
print(a)
for j in range(len(a)):
    count=0
    for i in range(len(pc)):
        if solute[0][f'pas_{i}_in_fli_{j}']==1:
            count=count+p[i]
    fli_cap.append(count) 

fli_cap

[209, 52]


[19, 33]

In [73]:
for i in range(len(p)):
 sum=0
 for j in range(len(a)):
  sum=sum+solute[0][f'pas_{i}_in_fli_{j}']
  if sum >1 :
    solute[0][f'pas_{i}_in_fli_{j}']=0


In [74]:
for j in range(len(a)):
 sum=0
 for i in range(len(p)):
  sum=sum+(solute[0][f'pas_{i}_in_fli_{j}'])*p[i]
  if sum >a[j] :
    solute[0][f'pas_{i}_in_fli_{j}']=0

In [75]:
no_pas=0
print(len(p))
for i in range(len(p)):
    for j in range(len(a)):
     if solute[0][f'pas_{i}_in_fli_{j}']==1:
        no_pas=no_pas+1
no_pas

15


15

In [76]:
fli_cap=[]
print(a)
for j in range(len(a)):
    count=0
    for i in range(len(pc)):
        if solute[0][f'pas_{i}_in_fli_{j}']==1:
            count=count+p[i]
    fli_cap.append(count) 

fli_cap

[209, 52]


[19, 33]

In [283]:
solute[0]

{'pas_0_in_fli_0': 0,
 'pas_0_in_fli_1': 1,
 'pas_10_in_fli_0': 1,
 'pas_10_in_fli_1': 0,
 'pas_11_in_fli_0': 1,
 'pas_11_in_fli_1': 0,
 'pas_12_in_fli_0': 0,
 'pas_12_in_fli_1': 1,
 'pas_13_in_fli_0': 1,
 'pas_13_in_fli_1': 0,
 'pas_14_in_fli_0': 1,
 'pas_14_in_fli_1': 0,
 'pas_1_in_fli_0': 1,
 'pas_1_in_fli_1': 0,
 'pas_2_in_fli_0': 0,
 'pas_2_in_fli_1': 1,
 'pas_3_in_fli_0': 0,
 'pas_3_in_fli_1': 1,
 'pas_4_in_fli_0': 1,
 'pas_4_in_fli_1': 0,
 'pas_5_in_fli_0': 0,
 'pas_5_in_fli_1': 1,
 'pas_6_in_fli_0': 0,
 'pas_6_in_fli_1': 1,
 'pas_7_in_fli_0': 0,
 'pas_7_in_fli_1': 1,
 'pas_8_in_fli_0': 0,
 'pas_8_in_fli_1': 1,
 'pas_9_in_fli_0': 1,
 'pas_9_in_fli_1': 0}

In [252]:
final_recloc_list = merge_sorted_table['RECLOC'].tolist()
final_pax_cnt=merge_sorted_table['PAX_CNT'].tolist()
final_seg_seq_cnt=merge_sorted_table['SEG_SEQ'].tolist()
final_score=merge_sorted_table['final_score'].tolist()

final_recloc_list

['BUHV67',
 'RHDO70',
 'PJSC24',
 'PSGX74',
 'MCNQ94',
 'ERZL70',
 'JISG46',
 'QXQZ78',
 'GQDX67',
 'SQMP46',
 'HQAO84',
 'ZTQP69',
 'WYWE97',
 'TWFD73',
 'YXHR85']

In [284]:
x.InventoryId

2    INV-ZZ-7767976
1    INV-ZZ-5004957
Name: InventoryId, dtype: object